In [1]:
import urllib.request
import pandas as pd
import numpy as np
import json
import os
import seaborn as sns

import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
from matplotlib import dates
import pingouin as pg
from scipy.stats import ttest_ind

sns.reset_defaults()
sns.set_theme(font_scale=0.7)
sns.set_theme(rc={'savefig.dpi':300})
# sns.set_theme(rc={"figure.dpi":300, 'savefig.dpi':300})

## Extraction of data to local

* *This part is for the data extraction part.* <br>
* *To open the link I used the urlopen with decode('ISO-8859-1') bcoz that's what let me download the data* <br>
* *For some links, there is fixed columns 56, but for some there is quite more than that. So the try except block handles that* <br>
* *For datasets that have columns greater than 56, I poped the last column and extracted the rest into a column and then build the dataframe from it.*
* *I have also created the json file for each of the metedata*


For Goddard

I see two folders under the Goddard Space Flight Center location. They are Pandora2s1 & Pandora32s1. I suppose they are readings taken from different Pandora instruments numbered 2 & 32 respectively. Pandora2s1 holds the bigger dataset [by size] & also extends back to longer dates [from 2021]. <br>

- For the NO2, So2, O3, Water Vapor I found the official data files. But for formaldehyde [HCHO] there is two files, but both of them are unvalidated: <br>
https://data.pandonia-global-network.org/GreenbeltMD/Pandora2s1/L2/Pandora2s1_GreenbeltMD_L2_rfus5p1-8.txt -> Formaldehyde data are unvalidated [start date 2021/08/24] <br>
https://data.pandonia-global-network.org/GreenbeltMD/Pandora2s1/L2/Pandora2s1_GreenbeltMD_L2_rfuh5p1-8.txt -> Formaldehyde data are unvalidated [start date 2021/09/15]<br>

In [ ]:
def getRawData(place, url_link_dict):
    for chemical, link in url_link_dict.items():
        with urllib.request.urlopen(link) as f:
            html = f.read().decode('ISO-8859-1')

        file_path_local = 'del.txt'
        with open(file_path_local, 'w') as f:
            f.write(html)

        with open(file_path_local, 'r', encoding='ISO-8859-1') as f:
            lines = f.readlines()    

        line_index_to_find = '---------------------------------------------------------------------------------------\n'
        line_index = [ind for ind, val in enumerate(lines) if val == line_index_to_find]

        metadata = dict()
        metadata_column_slices = lines[:line_index[0]]
        for column in metadata_column_slices:
            key, value = column.split(':')[0].strip(), column.split(':')[1].strip()
            metadata[key] = value

        chemical_metadata_path = f'{metadata_path}/{chemical}_{place}.json'
        with open(chemical_metadata_path, "w") as outfile: 
            json.dump(metadata, outfile)

        try: 
            colums = []
            columns_slices = lines[line_index[0]+1: line_index[1]]
            for column in columns_slices:
                col_name = column.split(':')[-1].strip()
                colums.append(col_name)

            final_values_list = []
            data_slices = lines[line_index[1]+1:]
            for data_row in data_slices:
                data_row = data_row.replace('\n', '')
                values = data_row.split()
                for i in range(1, len(values)):
                    values[i] = float(values[i])
                final_values_list.append(values)


            df = pd.DataFrame(final_values_list, 
                            columns = colums) 

            df.reset_index(inplace=True)
            csv_filepath = f'{data_folder_path}/{chemical}_{place}.csv'
            df.to_csv(csv_filepath, index=False)  
        except:
            colums = []
            columns_slices = lines[line_index[0]+1: line_index[1]]
            for column in columns_slices:
                col_name = column.split(':')[-1].strip()
                colums.append(col_name)
            colums.pop()

            final_values_list = []
            data_slices = lines[line_index[1]+1:]
            for data_row in data_slices:
                data_row = data_row.replace('\n', '')
                values = data_row.split()
                print(len(values))
                for i in range(1, len(values)):
                    values[i] = float(values[i])
                final_values_list.append(values[:len(colums)])

            df = pd.DataFrame(final_values_list, 
                            columns = colums) 

            df.reset_index(inplace=True)
            csv_filepath = f'{data_folder_path}/{chemical}_{place}.csv'
            df.to_csv(csv_filepath, index=False)           


data_folder_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data/Raw"
metadata_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data/Metadata"

# place = "BeltsVille"
# BeltsVille_link_dict = {
#     'NO2': 'https://data.pandonia-global-network.org/BeltsvilleMD/Pandora80s1/L2/Pandora80s1_BeltsvilleMD_L2_rnvs3p1-8.txt',
#     'HCHO': 'https://data.pandonia-global-network.org/BeltsvilleMD/Pandora80s1/L2/Pandora80s1_BeltsvilleMD_L2_rfus5p1-8.txt',
#     'SO2': 'https://data.pandonia-global-network.org/BeltsvilleMD/Pandora80s1/L2/Pandora80s1_BeltsvilleMD_L2_rsus1p1-8.txt',
#     'H2O': 'https://data.pandonia-global-network.org/BeltsvilleMD/Pandora80s1/L2/Pandora80s1_BeltsvilleMD_L2_rwvt1p1-8.txt',
#     'O3': 'https://data.pandonia-global-network.org/BeltsvilleMD/Pandora80s1/L2/Pandora80s1_BeltsvilleMD_L2_rout2p1-8.txt'
# }
# print(getRawData(place, BeltsVille_link_dict))


# place = "Goddard"
# Goddard_link_dict = {
#     'NO2': 'https://data.pandonia-global-network.org/GreenbeltMD/Pandora2s1/L2/Pandora2s1_GreenbeltMD_L2_rnvs3p1-8.txt',
#     'HCHO': 'https://data.pandonia-global-network.org/GreenbeltMD/Pandora2s1/L2/Pandora2s1_GreenbeltMD_L2_rfus5p1-8.txt',
#     'SO2': 'https://data.pandonia-global-network.org/GreenbeltMD/Pandora2s1/L2/Pandora2s1_GreenbeltMD_L2_rsus1p1-8.txt',
#     'H2O': 'https://data.pandonia-global-network.org/GreenbeltMD/Pandora2s1/L2/Pandora2s1_GreenbeltMD_L2_rwvt1p1-8.txt',
#     'O3': 'https://data.pandonia-global-network.org/GreenbeltMD/Pandora2s1/L2/Pandora2s1_GreenbeltMD_L2_rout2p1-8.txt'
# }
# print(getRawData(place, Goddard_link_dict))


# place = "Mcmillan"
# Mcmillan_link_dict = {
#     'NO2': 'https://data.pandonia-global-network.org/WashingtonDC/Pandora140s1/L2/Pandora140s1_WashingtonDC_L2_rnvs3p1-8.txt',
#     'HCHO': 'https://data.pandonia-global-network.org/WashingtonDC/Pandora140s1/L2/Pandora140s1_WashingtonDC_L2_rfus5p1-8.txt',
#     'SO2': 'https://data.pandonia-global-network.org/WashingtonDC/Pandora140s1/L2/Pandora140s1_WashingtonDC_L2_rsus1p1-8.txt',
#     'H2O': 'https://data.pandonia-global-network.org/WashingtonDC/Pandora140s1/L2/Pandora140s1_WashingtonDC_L2_rwvt1p1-8.txt',
#     'O3': 'https://data.pandonia-global-network.org/WashingtonDC/Pandora140s1/L2/Pandora140s1_WashingtonDC_L2_rout2p1-8.txt'
# }
# print(getRawData(place, Mcmillan_link_dict))

None


## Pre-processing of raw data into processed Data

* renaming long column names into short ones. [quality column, vertical amount column] --> name changed
* quality column codes changed into understandable form.

In [ ]:
def preprocessingDataset(place):
    raw_csv_path = {}
    for chemical in ["H2O", "HCHO", "NO2", "O3", "SO2"]:
        raw_csv_path[chemical] = f"{data_folder_path}/{chemical}_{place}.csv"

    for chemical, file in raw_csv_path.items():
        df = pd.read_csv(file)
        df.drop('index', axis=1, inplace=True)

        # rename of column names
        df.rename(columns={quality_column_dict[chemical]: 'quality', vertical_amount_column_dict[chemical]: 'vertical_amount'}, inplace=True) 
        
        ### renaming quality codes into understandable language. 10 -> high quality na, 11: 'medium quality na', 12: 'low quality na'
        quality_dict = {10: 'high quality na', 11: 'medium quality na', 12: 'low quality na'} 
        df['quality'] = df['quality'].map(quality_dict)

        ### Dealing with just 3 columms in here [datetime_UTC, quality, vertical_amount]
        #### Change to pandas dateTime object
        def change_iso8601_to_datetime_object(in_str):
            return datetime.datetime.strptime(in_str, '%Y%m%dT%H%M%S.%fZ')

        df['datetime_UTC'] = df['UT date and time for measurement center, yyyymmddThhmmssZ (ISO 8601)'].apply(change_iso8601_to_datetime_object) 
        ################################################################################
        csv_filepath = f'/Users/ujjawal/Documents/Work/PandoraWork_2025/Data/{chemical}_{place}.csv'
        df.to_csv(csv_filepath, index=False, date_format='%Y-%m-%d %H:%M:%S%z')        
    

quality_column_dict = {
    'NO2': 'L2 data quality flag for nitrogen dioxide, 0=assured high quality, 1=assured medium quality, 2=assured low quality, 10=not-assured high quality, 11=not-assured medium quality, 12=not-assured low quality, 20=unusable high quality, 21=unusable medium quality, 22=unusable low quality',
    'HCHO': 'L2 data quality flag for formaldehyde, 0=assured high quality, 1=assured medium quality, 2=assured low quality, 10=not-assured high quality, 11=not-assured medium quality, 12=not-assured low quality, 20=unusable high quality, 21=unusable medium quality, 22=unusable low quality',
    'SO2':  'L2 data quality flag for sulfur dioxide, 0=assured high quality, 1=assured medium quality, 2=assured low quality, 10=not-assured high quality, 11=not-assured medium quality, 12=not-assured low quality, 20=unusable high quality, 21=unusable medium quality, 22=unusable low quality', 
    'H2O': 'L2 data quality flag for water vapor, 0=assured high quality, 1=assured medium quality, 2=assured low quality, 10=not-assured high quality, 11=not-assured medium quality, 12=not-assured low quality, 20=unusable high quality, 21=unusable medium quality, 22=unusable low quality',
    'O3': 'L2 data quality flag for ozone, 0=assured high quality, 1=assured medium quality, 2=assured low quality, 10=not-assured high quality, 11=not-assured medium quality, 12=not-assured low quality, 20=unusable high quality, 21=unusable medium quality, 22=unusable low quality'
}

vertical_amount_column_dict = {
    'NO2': 'Nitrogen dioxide total vertical column amount [moles per square meter], -9e99=retrieval not successful',
    'HCHO': 'Formaldehyde total vertical column amount [moles per square meter], -9e99=retrieval not successful',
    'SO2': 'Sulfur dioxide total vertical column amount [moles per square meter], -9e99=retrieval not successful',
    'H2O': 'Water vapor total vertical column amount [moles per square meter], -9e99=retrieval not successful',
    'O3': 'Ozone total vertical column amount [moles per square meter], -9e99=retrieval not successful'
}

data_folder_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data/Raw"
for place in ['BeltsVille', 'Goddard', 'Mcmillan']:
    preprocessingDataset(place)
    # break

# Analysis

## Scatter plot and lineplot combined -> all years

In [ ]:
def processDataset(csvPath): # returns a df object
    '''
    Filtering: Three columns, High Quality Data
            @ get three column datatime_UTC, quality, vertical_amount
            @ filter data with just the high quality data

    - change datetime_UTC to EST
    - date, year, month, hour, time, rounded_hours
    - get the date extracted
    - get the year extracted
    - get the month extracted
    - get the hour extracted
    - get the time extracted 
    - get the rounded hours extracted
    ------> return the df with these things and do the analysis
    '''
    df = pd.read_csv(csvPath,  parse_dates=['datetime_UTC'])
    # Filtering
    workingColumns = ['datetime_UTC', 'quality', 'vertical_amount']
    df = df[workingColumns]
    df = df[df.quality == 'high quality na']
    #############################
    
    df['datetime_EST'] = df['datetime_UTC'].dt.tz_localize('UTC').dt.tz_convert('America/New_York') # change into EST
    # Extract date, year, month, hour, time, rounded_hours
    df.loc[:, 'Date'] = df['datetime_EST'].dt.date
    df.loc[:, 'Year'] = df['datetime_EST'].dt.year
    df.loc[:, 'Month'] = df['datetime_EST'].dt.month_name()
    df.loc[:, 'Hour'] = df['datetime_EST'].dt.hour
    df.loc[:, 'Time'] = df['datetime_EST'].dt.time

    # def round_to_nearest_hour(timestamp):

    #     if timestamp.minute >= 30 or (timestamp.minute == 29 and timestamp.second >= 30):
    #         # Round up
    #         rounded_timestamp = timestamp + timedelta(hours=1)
    #         rounded_timestamp = rounded_timestamp.replace(minute=0, second=0, microsecond=0)
    #     else:
    #         # Round down
    #         rounded_timestamp = timestamp.replace(minute=0, second=0, microsecond=0)
    #     return rounded_timestamp
    # df.loc[:, "roundedHour"] = df['datetime_EST'].apply(round_to_nearest_hour).dt.hour   
    return df

def scatterPlot_lineplot_combined(chemical, places_list):
    preprocessed_csv_path = {}
    for place in places_list:
        preprocessed_csv_path[place] = f"{data_folder_path}/{chemical}_{place}.csv"

    for place, path in preprocessed_csv_path.items():
        df = processDataset(path)
        df['MovingAvg'] = df['vertical_amount'].rolling(window=1000).mean()
        fig, axes = plt.subplots(2, 1, sharex=True, figsize=(10, 8))
        
        g = sns.lineplot(ax=axes[1], x="datetime_EST", y="vertical_amount", data=df, alpha=1, hue='Year', palette={2020:'darkgreen', 2021: 'pink', 2022: 'blue', 2023: 'red', 2024:'green', 2025:"lightyellow"})
        # g = sns.lineplot(ax=axes[1], x='datetime_EST', y='MovingAvg', data=df, color="black", label="Moving average of 1000 points")  # uncomment when the black line is not needed. 
        g = sns.scatterplot(ax=axes[0],x="datetime_EST", y="vertical_amount", data=df, alpha=0.6, s=20, color='black')

        # axes[1].xaxis.set_major_locator(dates.MonthLocator())
        axes[1].xaxis.set_major_formatter(dates.DateFormatter('%b'))
        axes[1].tick_params(axis='x', rotation=30)

        sec_xaxis = axes[1].secondary_xaxis(-0.09)
        sec_xaxis.xaxis.set_major_locator(dates.YearLocator())
        sec_xaxis.xaxis.set_major_formatter(dates.DateFormatter('%Y'))
        sec_xaxis.spines['bottom'].set_visible(False)

        g.set(xlabel='', 
        ylabel='Vertical Column amount [moles per square meter',
        title= f"{place} {chemical} Plot")

        axes[1].set(xlabel='')
        axes[1].set(ylabel='')

        axes[0].set(ylabel='')
        fig.supylabel(r"Vertical Column Amount [moles/m$\mathregular{^2}$]", fontsize=12)
        # plotSavePath = f"/Users/ujjawal/Documents/Work/PandoraWork_2025/Plots/{chemical}/scatterPlotLinePlotCombined_{place}.png"
        # plt.savefig(plotSavePath, bbox_inches='tight');
        plt.show() 
        break


# data_folder_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data"
# chemical = "H2O"
# places_list = ["BeltsVille", "Goddard", "Mcmillan"]
# scatterPlot_lineplot_combined(chemical, places_list)

data_folder_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data"
for chemical in ["H2O", "HCHO", "NO2", "O3", "SO2"]:
    places_list = ["BeltsVille", "Goddard", "Mcmillan"]
    scatterPlot_lineplot_combined(chemical, places_list)

## Yearly plot: Year & Places as a variable || with SE || Combined for all the places

* *For each month, it gives in a mean value of the vertical amount* <br>
* *For median, we can do this: estimator=np.median in `g = sns.lineplot()`*

In [ ]:
def processDataset(csvPath): # returns a df object
    '''
    Filtering: Three columns, High Quality Data
            @ get three column datatime_UTC, quality, vertical_amount
            @ filter data with just the high quality data

    - change datetime_UTC to EST
    - date, year, month, hour, time, rounded_hours
    - get the date extracted
    - get the year extracted
    - get the month extracted
    - get the hour extracted
    - get the time extracted 
    - get the rounded hours extracted
    ------> return the df with these things and do the analysis
    '''
    df = pd.read_csv(csvPath,  parse_dates=['datetime_UTC'])
    # Filtering
    workingColumns = ['datetime_UTC', 'quality', 'vertical_amount']
    df = df[workingColumns]
    df = df[df.quality == 'high quality na']
    #############################
    
    df['datetime_EST'] = df['datetime_UTC'].dt.tz_localize('UTC').dt.tz_convert('America/New_York') # change into EST
    # Extract date, year, month, hour, time, rounded_hours
    df.loc[:, 'Date'] = df['datetime_EST'].dt.date
    df.loc[:, 'Year'] = df['datetime_EST'].dt.year
    df.loc[:, 'Month'] = df['datetime_EST'].dt.month_name()
    df.loc[:, 'Hour'] = df['datetime_EST'].dt.hour
    df.loc[:, 'Time'] = df['datetime_EST'].dt.time

    # def round_to_nearest_hour(timestamp):

    #     if timestamp.minute >= 30 or (timestamp.minute == 29 and timestamp.second >= 30):
    #         # Round up
    #         rounded_timestamp = timestamp + timedelta(hours=1)
    #         rounded_timestamp = rounded_timestamp.replace(minute=0, second=0, microsecond=0)
    #     else:
    #         # Round down
    #         rounded_timestamp = timestamp.replace(minute=0, second=0, microsecond=0)
    #     return rounded_timestamp
    # df.loc[:, "roundedHour"] = df['datetime_EST'].apply(round_to_nearest_hour).dt.hour   
    return df


def comparisonYearlyPlotWithSE(chemical, year, places_list):
    preprocessed_csv_path = {}
    for place in places_list:
        preprocessed_csv_path[place] = f"{data_folder_path}/{chemical}_{place}.csv"
    
    df_list = {}
    for place, path in preprocessed_csv_path.items():
        df = processDataset(path)
        df = df[df.Year == year]
        df_list[place] = df

    fig, ax = plt.subplots(figsize=(10,8))
    for place, df in df_list.items():
        g = sns.lineplot(x=df['Month'], y=df['vertical_amount'], ax=ax, err_style="bars", errorbar=("se"), c=color_dict[place], label=place)

    ax.tick_params(axis='x', rotation=30)
    g.set(xlabel='', 
        ylabel=r"Vertical Column Amount [moles/m$\mathregular{^2}$]",
        title= f'{chemical} column amount comparison for year {year}')
    if chemical == "SO2":
        plt.ticklabel_format(style='plain', axis='y',useOffset=False)
    plotSavePath = f"/Users/ujjawal/Documents/Work/PandoraWork_2025/Plots/{chemical}/Comparision of {year}.png"
    plt.savefig(plotSavePath, bbox_inches='tight');
    plt.show();

color_dict = {'BeltsVille': 'green',
              'Goddard': 'blue',
              'Mcmillan':'red'}


# data_folder_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data"
# places_list = ["BeltsVille", "Goddard", "Mcmillan"]
# for chemical in ["H2O","HCHO", "NO2", "O3", "SO2"]:
#     for year in [2023, 2024]:
#         comparisonYearlyPlotWithSE(chemical, year, places_list)

data_folder_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data"
places_list = ["BeltsVille", "Goddard", "Mcmillan"]
for chemical in ["SO2"]:
    for year in [2023, 2024]:
        comparisonYearlyPlotWithSE(chemical, year, places_list)

### Annual Trend -> Yearly plots on top of each other

In [ ]:
def processDataset(csvPath): # returns a df object
    df = pd.read_csv(csvPath,  parse_dates=['datetime_UTC'])
    # Filtering
    workingColumns = ['datetime_UTC', 'quality', 'vertical_amount']
    df = df[workingColumns]
    df = df[df.quality == 'high quality na']
    
    df['datetime_EST'] = df['datetime_UTC'].dt.tz_localize('UTC').dt.tz_convert('America/New_York') # change into EST
    # Extract date, year, month, hour, time, rounded_hours
    df.loc[:, 'Date'] = df['datetime_EST'].dt.date
    df.loc[:, 'Year'] = df['datetime_EST'].dt.year
    df.loc[:, 'Month'] = df['datetime_EST'].dt.month_name()
    df.loc[:, 'Hour'] = df['datetime_EST'].dt.hour
    df.loc[:, 'Time'] = df['datetime_EST'].dt.time 
    return df


def annualTrend(chemical, places_list, year_list):
    preprocessed_csv_path = {}
    for place in places_list:
        preprocessed_csv_path[place] = f"{data_folder_path}/{chemical}_{place}.csv"
    
    df_list_year = {} # year: df_year
    for place, path in preprocessed_csv_path.items():
        df_big = processDataset(path)
        for year in year_list:
            df = df_big[df_big.Year == year]
            df_list_year[year] = df

    fig, ax = plt.subplots(figsize=(10,8))
    for year, df in df_list_year.items():
        # g = sns.lineplot(x=df['Month'], y=df['vertical_amount'], ax=ax, err_style="bars", errorbar=("se"), linestyle = linestyle_year[year], label=year)
        g = sns.lineplot(x=df['Month'], y=df['vertical_amount'], ax=ax, err_style="bars", errorbar=("se"), c = color_dict[year], label=year)

    ax.tick_params(axis='x', rotation=30)
    ax.set(ylim=(0, 0.00006))

    g.set(xlabel='', 
        ylabel=r"Vertical Column Amount [moles/m$\mathregular{^2}$]",
        title= f'{chemical} Annual trend across {place} {year_list}')
    if chemical == "SO2":
        plt.ticklabel_format(style='plain', axis='y',useOffset=False)
    
    # plotSavePath = f"/Users/ujjawal/Documents/Work/PandoraWork_2025/Plots/{chemical}/Annual trend {place}.png"
    # plt.savefig(plotSavePath, bbox_inches='tight');
    plt.show();


color_dict = { 2021:"red",
               2022: "blue",
               2023:"green",
               2024:"black"
               }
linestyle_year = {2021:"-",
                  2022:"--",
                  2023:":",
                  2024:"-.",}


# data_folder_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data"
# places_list = ["Mcmillan"]
# year_list = [2021, 2022, 2023, 2024]
# for chemical in ["SO2"]:
#         annualTrend(chemical, places_list, year_list)


# data_folder_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data"
# places_list = ["BeltsVille"]
# year_list = [2023]
# for chemical in ["SO2"]:
#         annualTrend(chemical, places_list, year_list)

data_folder_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data"
places_list = ["Goddard"]
year_list = [2022, 2023]
for chemical in ["SO2"]:
        annualTrend(chemical, places_list, year_list)

## Comparison: Data Distribution & One-to-one line

##### Data Distribution

In [ ]:
def processDataset(csvPath): # returns a df object
    '''
    Filtering: Three columns, High Quality Data
            @ get three column datatime_UTC, quality, vertical_amount
            @ filter data with just the high quality data

    - change datetime_UTC to EST
    - date, year, month, hour, time, rounded_hours
    - get the date extracted
    - get the year extracted
    - get the month extracted
    - get the hour extracted
    - get the time extracted 
    - get the rounded hours extracted
    ------> return the df with these things and do the analysis
    '''
    df = pd.read_csv(csvPath,  parse_dates=['datetime_UTC'])
    # Filtering
    workingColumns = ['datetime_UTC', 'quality', 'vertical_amount']
    df = df[workingColumns]
    df = df[df.quality == 'high quality na']
    #############################
    
    df['datetime_EST'] = df['datetime_UTC'].dt.tz_localize('UTC').dt.tz_convert('America/New_York') # change into EST
    # Extract date, year, month, hour, time, rounded_hours
    df.loc[:, 'Date'] = df['datetime_EST'].dt.date
    df.loc[:, 'Year'] = df['datetime_EST'].dt.year
    df.loc[:, 'Month'] = df['datetime_EST'].dt.month_name()
    df.loc[:, 'Hour'] = df['datetime_EST'].dt.hour
    df.loc[:, 'Time'] = df['datetime_EST'].dt.time

    # def round_to_nearest_hour(timestamp):

    #     if timestamp.minute >= 30 or (timestamp.minute == 29 and timestamp.second >= 30):
    #         # Round up
    #         rounded_timestamp = timestamp + timedelta(hours=1)
    #         rounded_timestamp = rounded_timestamp.replace(minute=0, second=0, microsecond=0)
    #     else:
    #         # Round down
    #         rounded_timestamp = timestamp.replace(minute=0, second=0, microsecond=0)
    #     return rounded_timestamp
    # df.loc[:, "roundedHour"] = df['datetime_EST'].apply(round_to_nearest_hour).dt.hour   
    return df

def dataDistribution(chemical, year, places_list, t_test_places):
    preprocessed_csv_path = {}
    for place in places_list:
        preprocessed_csv_path[place] = f"{data_folder_path}/{chemical}_{place}.csv"
    
    df_list = {}
    for place, path in preprocessed_csv_path.items():
        df = processDataset(path)
        if year == "all":
            pass
        else:
            df = df[df.Year == year]
        df_list[place] = df
    
    fig, axes = plt.subplots(len(places_list), 1, sharey=True, sharex=True, figsize=(16, 9))
    ax = axes.flatten()
    index = 0
    
    for place, df in df_list.items():
        df_mean = df['vertical_amount'].mean()
        df_median = df['vertical_amount'].median()
        df_std = df['vertical_amount'].std()

        g = sns.histplot(data = df, x = "vertical_amount", kde = True, ax = ax[index])
        g.axvline(df_mean, color='k', linestyle=':', label='Mean')
        g.axvline(df_median, color='r', linestyle='--', label='Median')
        g.axvline(df_mean + df_std, color='g', linestyle='-', label='Mean + 1 SD')
        g.axvline(df_mean - df_std, color='g', linestyle='-', label='Mean - 1 SD')
        g.set_title(place)
        g.legend()
        index += 1

    for x in ax:
        x.set(xlabel='')
        x.set(ylabel='')

    ## t-test value for the things ###########################
    x_axis_val = (g.get_xlim()[0] + g.get_xlim()[1]) / 2
    y_axis_val = 2000
    # x_axis_val, y_axis_val = 7000, 2000
    for t_test_combination in t_test_places:
        place1_verticalAmt_numpy = df_list[t_test_combination[0]]["vertical_amount"].to_numpy()
        place2_verticalAmt_numpy = df_list[t_test_combination[1]]["vertical_amount"].to_numpy()
        
        t_stat, p_val = ttest_ind(place1_verticalAmt_numpy, place2_verticalAmt_numpy) 
        ax[1].text(x_axis_val, y_axis_val,f'{t_test_combination[0]} & {t_test_combination[1]} p-val: {"{:.2e}".format(p_val)}', fontsize=12) 
        y_axis_val -= 1500

    fig.suptitle(f'{chemical} data points distributution for {year}')
    fig.supxlabel(r"Vertical Column Amount [moles/m$\mathregular{^2}$]")
    fig.supylabel('Frequency Count')
    
    plotSavePath = f"/Users/ujjawal/Documents/Work/PandoraWork_2025/Plots/{chemical}/Data points distributution for {year}.png"
    plt.savefig(plotSavePath, bbox_inches='tight');
    plt.show()


data_folder_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data"
# chemical = "H2O"
# year = "all"
# places_list = ["BeltsVille", "Goddard", "Mcmillan"]
# t_test_places = [["Goddard", "Mcmillan"], ["Goddard", "BeltsVille"]]
# dataDistribution(chemical, year, places_list, t_test_places)


places_list = ["BeltsVille", "Goddard", "Mcmillan"]
t_test_places = [["Goddard", "Mcmillan"], ["Goddard", "BeltsVille"]]

for chemical in ["H2O", "HCHO", "NO2", "O3", "SO2"]:
        for year in ["all", 2023, 2024]:
            dataDistribution(chemical, year, places_list, t_test_places)

##### One-to-one line comparison

In [ ]:
def add_identity(axes, *line_args, **line_kwargs):
    identity, = axes.plot([], [], *line_args, **line_kwargs)
    def callback(axes):
        low_x, high_x = axes.get_xlim()
        low_y, high_y = axes.get_ylim()
        low = max(low_x, low_y)
        high = min(high_x, high_y)
        identity.set_data([low, high], [low, high])
    callback(axes)
    axes.callbacks.connect('xlim_changed', callback)
    axes.callbacks.connect('ylim_changed', callback)
    return axes

def processDataset(csvPath): # returns a df object
    '''
    Filtering: Three columns, High Quality Data
            @ get three column datatime_UTC, quality, vertical_amount
            @ filter data with just the high quality data

    - change datetime_UTC to EST
    - date, year, month, hour, time, rounded_hours
    - get the date extracted
    - get the year extracted
    - get the month extracted
    - get the hour extracted
    - get the time extracted 
    - get the rounded hours extracted
    ------> return the df with these things and do the analysis
    '''
    df = pd.read_csv(csvPath,  parse_dates=['datetime_UTC'])
    # Filtering
    workingColumns = ['datetime_UTC', 'quality', 'vertical_amount']
    df = df[workingColumns]
    df = df[df.quality == 'high quality na']
    #############################
    
    df['datetime_EST'] = df['datetime_UTC'].dt.tz_localize('UTC').dt.tz_convert('America/New_York') # change into EST
    # Extract date, year, month, hour, time, rounded_hours
    df.loc[:, 'Date'] = df['datetime_EST'].dt.date
    df.loc[:, 'Year'] = df['datetime_EST'].dt.year
    df.loc[:, 'Month'] = df['datetime_EST'].dt.month_name()
    df.loc[:, 'Hour'] = df['datetime_EST'].dt.hour
    df.loc[:, 'Time'] = df['datetime_EST'].dt.time

    df = df[['datetime_EST', 'vertical_amount', 'Year']]
    df.set_index('datetime_EST', inplace=True)
    df = df.resample('15T').mean()
    df = df[df['vertical_amount'].notna()]
    return df

def one_to_one_comparison(chemical, year, places_list):
    preprocessed_csv_path = {}
    for place in places_list:
        preprocessed_csv_path[place] = f"{data_folder_path}/{chemical}_{place}.csv"
    
    df_list = {}
    for place, path in preprocessed_csv_path.items():
        df = processDataset(path)
        if year == "all":
            pass
        else:
            df = df[df.Year == year]
        df_list[place] = df
    
    place1 = df_list[places_list[0]]
    place2 = df_list[places_list[1]]

    common_time_df = pd.merge(place1, place2, left_index=True, right_index=True, suffixes=(f'_{places_list[0]}', f'_{places_list[1]}'))

    fig, ax = plt.subplots(figsize=(10, 8))
    g = sns.scatterplot(ax=ax,
                        x = f"vertical_amount_{places_list[0]}",
                        y = f"vertical_amount_{places_list[1]}",
                        data= common_time_df)
    
    ## draw the diagnoal line
    add_identity(ax, color='green', linewidth=4, label='one-to-one line')

    # best fit line
    plt.plot(np.unique(common_time_df[f'vertical_amount_{places_list[0]}']), 
        np.poly1d(np.polyfit(common_time_df[f'vertical_amount_{places_list[0]}'], 
        common_time_df[f'vertical_amount_{places_list[1]}'], 1))(np.unique(common_time_df[f'vertical_amount_{places_list[0]}'])),
        color='black', linewidth=4, ls='--', label='best fit line')

    plot_title = f'{chemical} {places_list[0]} vs {places_list[1]} comparison for {year} year'
    g.set(xlabel= f'{places_list[0]} Vertical Column amount [moles/meter sq.]', 
        ylabel= f'{places_list[1]} Vertical Column amount [moles/meter sq.]',
        title= plot_title)
    plt.legend()

    plotSavePath = f"/Users/ujjawal/Documents/Work/PandoraWork_2025/Plots/{chemical}/1-1 comparison {places_list[0]} vs {places_list[1]} for {year} year.png"
    plt.savefig(plotSavePath, bbox_inches='tight');

    plt.show();    
    

data_folder_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data"
# chemical = "H2O"
# year = "all"
# places_list = ["Goddard", "Mcmillan"]
# # places_list =["Goddard", "BeltsVille"]
# # places_list = ["Mcmillan", "BeltsVille"]
# for places_list in [["Goddard", "Mcmillan"], ["Goddard", "BeltsVille"], ["Mcmillan", "BeltsVille"]]:
#     one_to_one_comparison(chemical, year, places_list)


for places_list in [["Goddard", "Mcmillan"], ["Goddard", "BeltsVille"], ["Mcmillan", "BeltsVille"]]:
    for chemical in ["H2O", "HCHO", "NO2", "O3", "SO2"]:
        for year in ["all", 2023, 2024]:
            one_to_one_comparison(chemical, year, places_list)

## Hourly Plot

In [ ]:
def processDataset(csvPath): # returns a df object
    season_dict = {'January':'Winter',
                'February':'Winter',
                'March':'Winter',
                'April':'Spring',
                'May':'Spring',
                'June':'Spring',
                'July':'Summer',
                'August':'Summer',
                'September':'Summer',
                'October':'Fall',
                'November':'Fall',
                'December':'Fall'}

    df = pd.read_csv(csvPath,  parse_dates=['datetime_UTC'])
    # Filtering
    workingColumns = ['datetime_UTC', 'quality', 'vertical_amount']
    df = df[workingColumns]
    df = df[df.quality == 'high quality na']
    #############################
    
    df['datetime_EST'] = df['datetime_UTC'].dt.tz_localize('UTC').dt.tz_convert('America/New_York') # change into EST
    # Extract date, year, month, hour, time, rounded_hours
    df.loc[:, 'Date'] = df['datetime_EST'].dt.date
    df.loc[:, 'Year'] = df['datetime_EST'].dt.year
    df.loc[:, 'Month'] = df['datetime_EST'].dt.month_name()
    df.loc[:, 'Hour'] = df['datetime_EST'].dt.hour
    df.loc[:, 'Time'] = df['datetime_EST'].dt.time
    df['Season'] = df['Month'].map(season_dict)

    def round_to_nearest_hour(timestamp):

        if timestamp.minute >= 30 or (timestamp.minute == 29 and timestamp.second >= 30):
            # Round up
            rounded_timestamp = timestamp + timedelta(hours=1)
            rounded_timestamp = rounded_timestamp.replace(minute=0, second=0, microsecond=0)
        else:
            # Round down
            rounded_timestamp = timestamp.replace(minute=0, second=0, microsecond=0)
        return rounded_timestamp
    df.loc[:, "roundedHour"] = df['datetime_EST'].apply(round_to_nearest_hour).dt.hour   
    return df

def hourlyPlot(chemical, year, places_list):
    preprocessed_csv_path = {}
    for place in places_list:
        preprocessed_csv_path[place] = f"{data_folder_path}/{chemical}_{place}.csv"
    
    df_list = {}
    for place, path in preprocessed_csv_path.items():
        df = processDataset(path)
        if year == "all":
            pass
        else:
            df = df[df.Year == year]
        df_list[place] = df
        
    for place, df in df_list.items():
        fig, axes = plt.subplots(2, 2, sharey=True, figsize=(15, 10))
        ax = axes.flatten()
        for season in df.Season.unique():
            df_particular_season_filterd = df[df.Season == season]
            g = sns.boxplot(x='roundedHour', 
                            y='vertical_amount', 
                            data=df_particular_season_filterd, 
                            ax= ax[season_axes_dict[season]], 
                            color=color_dict[season])
            ax[season_axes_dict[season]].set_title(season)
        
        # for removing the labels for each subplots
        for x in ax:
            x.set(xlabel='')
            x.set(ylabel='')
        
        # whole subplots figures 
        fig.suptitle(f'{place} {chemical} Hourly plots for {year} year')
        fig.supxlabel('EST Hours')
        fig.supylabel(r"Vertical Column Amount [moles/m$\mathregular{^2}$]")
        
        plotSavePath = f"/Users/ujjawal/Documents/Work/PandoraWork_2025/Plots/{chemical}/Hourly plot {place} for {year} year.png"
        plt.savefig(plotSavePath, bbox_inches='tight');
        plt.show()  
    

season_axes_dict = {'Winter': 0, 
                    'Spring': 1, 
                    'Summer':2, 
                    'Fall': 3}

color_dict = {'Winter': 'lightblue',
              'Spring': 'yellow',
              'Summer':'red',
              'Fall': 'lightgreen'}

# places_list = ["BeltsVille", "Goddard", "Mcmillan"]
# chemical = "H2O"
# year = "all"
# hourlyPlot(chemical, year, places_list)    


# places_list = ["BeltsVille", "Goddard", "Mcmillan"]
# chemical = "H2O"
# year = 2023
# hourlyPlot(chemical, year, places_list) 

# places_list = ["BeltsVille", "Goddard", "Mcmillan"]
# chemical = "H2O"
# year = 2024
# hourlyPlot(chemical, year, places_list) 

places_list = ["BeltsVille", "Goddard", "Mcmillan"]
for chemical in ["HCHO", "NO2", "O3", "SO2"]:
    for year in ["all", 2023, 2024]:
        hourlyPlot(chemical, year, places_list) 

## Seasonality Plot

In [ ]:
def processDataset(csvPath): # returns a df object
    season_dict = {'January':'Winter',
                'February':'Winter',
                'March':'Winter',
                'April':'Spring',
                'May':'Spring',
                'June':'Spring',
                'July':'Summer',
                'August':'Summer',
                'September':'Summer',
                'October':'Fall',
                'November':'Fall',
                'December':'Fall'}

    df = pd.read_csv(csvPath,  parse_dates=['datetime_UTC'])
    # Filtering
    workingColumns = ['datetime_UTC', 'quality', 'vertical_amount']
    df = df[workingColumns]
    df = df[df.quality == 'high quality na']
    #############################
    
    df['datetime_EST'] = df['datetime_UTC'].dt.tz_localize('UTC').dt.tz_convert('America/New_York') # change into EST
    # Extract date, year, month, hour, time, rounded_hours
    df.loc[:, 'Date'] = df['datetime_EST'].dt.date
    df.loc[:, 'Year'] = df['datetime_EST'].dt.year
    df.loc[:, 'Month'] = df['datetime_EST'].dt.month_name()
    df.loc[:, 'Hour'] = df['datetime_EST'].dt.hour
    df.loc[:, 'Time'] = df['datetime_EST'].dt.time
    df['Season'] = df['Month'].map(season_dict)

    def round_to_nearest_hour(timestamp):

        if timestamp.minute >= 30 or (timestamp.minute == 29 and timestamp.second >= 30):
            # Round up
            rounded_timestamp = timestamp + timedelta(hours=1)
            rounded_timestamp = rounded_timestamp.replace(minute=0, second=0, microsecond=0)
        else:
            # Round down
            rounded_timestamp = timestamp.replace(minute=0, second=0, microsecond=0)
        return rounded_timestamp
    df.loc[:, "roundedHour"] = df['datetime_EST'].apply(round_to_nearest_hour).dt.hour   
    return df

def seasonalityVariation(chemical, year, places_list):
    preprocessed_csv_path = {}
    for place in places_list:
        preprocessed_csv_path[place] = f"{data_folder_path}/{chemical}_{place}.csv"
    
    df_list = {}
    for place, path in preprocessed_csv_path.items():
        df = processDataset(path)
        if year == "all":
            pass
        else:
            df = df[df.Year == year]
        df_list[place] = df
    
    fig, axes = plt.subplots(2, 2, sharey=True, sharex=True, figsize=(20, 15))
    ax = axes.flatten()
    index = 0        
    for place, df in df_list.items():
        g = sns.boxplot(data = df, x = "Season", y = 'vertical_amount', showmeans = True, ax = ax[index])
        g.set_title(place)
        g.legend()
        index += 1
    
    for x in ax:
        x.set(xlabel='')
        x.set(ylabel='')  
        # x.set_ylim(0, 8000)  
    
    fig.suptitle(f'Season Comparison {chemical} for {year} year')
    fig.supylabel(r"Vertical Column Amount [moles/m$\mathregular{^2}$]")

    plotSavePath = f"/Users/ujjawal/Documents/Work/PandoraWork_2025/Plots/{chemical}/Season comparison for {year} year.png"
    plt.savefig(plotSavePath, bbox_inches='tight');

    plt.show()

# places_list = ["BeltsVille", "Goddard", "Mcmillan"]
# chemical = "H2O"
# year = 2023
# seasonalityVariation(chemical, year, places_list) 

# places_list = ["BeltsVille", "Goddard", "Mcmillan"]
# chemical = "H2O"
# year = 2024
# seasonalityVariation(chemical, year, places_list) 

# places_list = ["BeltsVille", "Goddard", "Mcmillan"]
# chemical = "H2O"
# year = "all"
# seasonalityVariation(chemical, year, places_list) 


places_list = ["BeltsVille", "Goddard", "Mcmillan"]
for chemical in ["HCHO", "NO2", "O3", "SO2"]:
    for year in ["all", 2023, 2024]:
        seasonalityVariation(chemical, year, places_list) 

## Corelation Analysis

* I did a correlation analysis for Water vapor and Fomaldeyde to know what is the thing that is going on with this. 

In [ ]:
def processDataset(csvPath): # returns a df object
    df = pd.read_csv(csvPath,  parse_dates=['datetime_UTC'])
    # Filtering
    workingColumns = ['datetime_UTC', 'quality', 'vertical_amount']
    df = df[workingColumns]
    df = df[df.quality == 'high quality na']
    #############################
    
    df['datetime_EST'] = df['datetime_UTC'].dt.tz_localize('UTC').dt.tz_convert('America/New_York') # change into EST
    # Extract date, year, month, hour, time, rounded_hours
    df.loc[:, 'Date'] = df['datetime_EST'].dt.date
    df.loc[:, 'Year'] = df['datetime_EST'].dt.year
    df.loc[:, 'Month'] = df['datetime_EST'].dt.month_name()
    df.loc[:, 'Hour'] = df['datetime_EST'].dt.hour
    df.loc[:, 'Time'] = df['datetime_EST'].dt.time

    df = df[['datetime_EST', 'vertical_amount', 'Year']]
    df.set_index('datetime_EST', inplace=True)
    df = df.resample('15T').mean()
    df = df[df['vertical_amount'].notna()]
    return df

def corelation_value(year, place, chemicals):
    preprocessed_csv_path = {}
    for chemical in chemicals:
        preprocessed_csv_path[chemical] = f"{data_folder_path}/{chemical}_{place}.csv"
    
    df_list = {}
    for chemical, path in preprocessed_csv_path.items():
        df = processDataset(path)
        if year == "all":
            pass
        else:
            df = df[df.Year == year]
        df_list[chemical] = df
    
    chemical0 = chemicals[0]
    chemical1 = chemicals[1]

    df_H2O = df_list[chemical0]
    df_HCHO = df_list[chemical1]
    merged = pd.merge(df_H2O, df_HCHO, left_index=True, right_index=True, how='inner', suffixes=(f'_{chemical0}', f'_{chemical1}'))
    
    fig, ax = plt.subplots(figsize=(10, 8))
    g = sns.scatterplot(ax=ax,
                        x = f"vertical_amount_{chemical0}",
                        y = f"vertical_amount_{chemical1}",
                        data= merged)
    
    correlation = merged[f"vertical_amount_{chemical0}"].corr(merged[f"vertical_amount_{chemical1}"])
    ax.text(0.3, 0.8,
            f"Pearson Correlation: {correlation:.2f}",
            fontsize=20, 
            ha='center', va='center', transform=ax.transAxes) 

    # best fit line
    plt.plot(np.unique(merged[f"vertical_amount_{chemical0}"]), 
        np.poly1d(np.polyfit(merged[f"vertical_amount_{chemical0}"], 
        merged[f"vertical_amount_{chemical1}"], 1))(np.unique(merged[f"vertical_amount_{chemical0}"])),
        color='black', linewidth=4, ls='--', label='best fit line')

    plot_title = f'Comparison of {chemical0} vs {chemical1} in {place} for {year}'


    x_label = "moles/m$\\mathregular{^2}$"
    y_label = "moles/m$\\mathregular{^2}$"

    g.set(xlabel= f"{place} {chemical0} Column Amount [{x_label}]", 
        ylabel= f"{place} {chemical1} Column Amount [{y_label}]",
        title= plot_title)
    
    plt.legend()
    
    plotSavePath = f"/Users/ujjawal/Documents/Work/PandoraWork_2025/Plots/allCommon/{chemical0} vs {chemical1} {place} {year}.png"
    plt.savefig(plotSavePath, bbox_inches='tight');
    plt.show(); 

# For H2O vs HCHO
'''
# data_folder_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data"
# place = "Mcmillan"
# chemicals = ["H2O", "HCHO"]
# for year in [2021, 2022, 2023, 2024]:
#     corelation_value(year, place, chemicals)


# data_folder_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data"
# place = "BeltsVille"
# chemicals = ["H2O", "HCHO"]
# for year in [2023]:
#     corelation_value(year, place, chemicals)
'''


# For H2O vs SO2
'''
data_folder_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data"
place = "Mcmillan"
chemicals = ["H2O", "SO2"]
for year in [2021, 2022, 2023, 2024]:
    corelation_value(year, place, chemicals)


data_folder_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data"
place = "BeltsVille"
chemicals = ["H2O", "SO2"]
for year in [2023]:
    corelation_value(year, place, chemicals)
'''


# For HCHO vs SO2
data_folder_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data"
place = "Mcmillan"
chemicals = ["HCHO", "SO2"]
for year in [2021, 2022, 2023, 2024]:
    corelation_value(year, place, chemicals)


data_folder_path = "/Users/ujjawal/Documents/Work/PandoraWork_2025/Data"
place = "Goddard"
chemicals = ["HCHO", "SO2"]
for year in [2022, 2023]:
    corelation_value(year, place, chemicals)
